# Ensemble
* Stacking
    1) BaseDataset
    2) MSBPDataset
    3) MyAugmentDataset (CenterCrop)
* Soft Voting & Hard Voting: 코드 미완


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import glob

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 무지성 가보자고

* dataset별로 stacking 후 다시 hard voting?

### 1. BaseDataset - stacking

* val_set을 통해 학습

In [72]:
outputs = glob.glob('baseline/train_output/*')
outputs = [file for file in outputs if not file.endswith("msbp.csv")]
print(outputs)

['baseline/train_output/resnet18_e30.csv', 'baseline/train_output/densenet_e30.csv']


In [73]:
df = pd.read_csv(outputs[0], names=['label','pred'])
len(df)

3001

In [8]:
# 각 모델의 decision 합치기 => new_input
def create_new_data(outputs):
    new_input = []
    for output in outputs:
        df = pd.read_csv(output, names=['label','pred'])
        new_input.append(list(df['pred']))
    new_input = np.transpose(new_input)
    return new_input

In [74]:
base_new_input = create_new_data(outputs)
base_new_input.shape

(3001, 2)

In [75]:
base_new_input[:10]

array([[ 1,  1],
       [16, 16],
       [ 3,  3],
       [10,  4],
       [16, 16],
       [15, 15],
       [ 3,  3],
       [ 3,  3],
       [ 5,  4],
       [ 1,  1]])

In [12]:
df = pd.read_csv(outputs[0], names=['label','pred'])
labels = np.array(df['label'])
print(labels)

[ 0 16  3 ... 12  1  0]


In [13]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

base_lgbm = LGBMClassifier()

base_lgbm.fit(base_new_input, labels)
lgbm_pred = base_lgbm.predict(base_new_input)

print(accuracy_score(labels, lgbm_pred))

0.8247250916361213


In [14]:
def print_acc(outputs, labels):
    for model in outputs:
        model_name = model.split('/')[-1].split('.')[0]
        df = pd.read_csv(model, names=['label','pred'])
        pred = np.array(df['pred'])
        print(f'{model_name} 정확도:', accuracy_score(labels, pred))

In [15]:
print_acc(outputs, labels)

resnet18_e30 정확도: 0.696434521826058
densenet_e30 정확도: 0.7910696434521826


* inference - base_lgbm의 inference

In [16]:
# 같은 이름(모델)의 inference 결과 path 가져오기
def path2inf(outputs):
    inf_outputs = []
    for model in outputs:
        model_name = model.split('/')[-1].split('.')[0]
        inf_outputs.append('baseline/output/'+model_name+'.csv')
    return inf_outputs

In [46]:
inf_outputs = path2inf(outputs)
print(inf_outputs)

['baseline/output/resnet18_e30.csv', 'baseline/output/densenet_e30.csv']


In [18]:
df = pd.read_csv(inf_outputs[0])
df.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,6
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12


In [19]:
def create_inf_data(outputs):
    new_input = []
    for output in outputs:
        df = pd.read_csv(output)
        new_input.append(list(df['ans']))
    new_input = np.transpose(new_input)
    return new_input

In [47]:
base_inf_input = create_inf_data(inf_outputs)
base_inf_input.shape

(12600, 2)

In [48]:
base_inf_input[:10]

array([[13, 13],
       [ 1,  5],
       [ 6, 13],
       [13, 13],
       [12, 12],
       [ 0,  0],
       [ 1,  1],
       [10,  4],
       [ 4,  1],
       [ 3,  3]])

In [22]:
base_lgbm_inf = base_lgbm.predict(base_inf_input)

In [24]:
# basedataset으로만 구한 결과 csv
base_info = pd.read_csv('input/data/eval/info.csv')
base_info['ans'] = base_lgbm_inf
base_info.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12


In [125]:
base_info.to_csv('base_info.csv', index=False)

### 2) MSBP - stacking

* 학습

In [76]:
outputs = glob.glob('baseline/train_output/*')
outputs = [file for file in outputs if file.endswith("msbp.csv")]
print(outputs)

['baseline/train_output/densenet_e30_msbp.csv', 'baseline/train_output/resnet_e30_msbp.csv', 'baseline/train_output/resnet_e30_adam_msbp.csv']


In [77]:
df = pd.read_csv(outputs[0], names=['label','pred'])
df.head()

,label,pred
0,0,1
1,0,0
2,12,12
3,0,0
4,0,0


In [79]:
msbp_new_input = create_new_data(outputs)
msbp_new_input.shape

(3001, 3)

In [80]:
msbp_new_input[:10]

array([[ 1,  1,  1],
       [ 0,  0,  0],
       [12, 12,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 6,  0,  0],
       [ 6,  0,  0],
       [ 0,  0,  7]])

In [29]:
df = pd.read_csv(outputs[0], names=['label','pred'])
labels = np.array(df['label'])
print(labels)

[ 0  0 12 ...  1  1  1]


In [33]:
msbp_lgbm = LGBMClassifier()

msbp_lgbm.fit(msbp_new_input, labels)
msbp_lgbm_pred = msbp_lgbm.predict(msbp_new_input)

print(accuracy_score(labels, msbp_lgbm_pred))

0.8153948683772076


In [34]:
print_acc(outputs, labels)

densenet_e30_msbp 정확도: 0.7560813062312562
resnet_e30_msbp 정확도: 0.639786737754082
resnet_e30_adam_msbp 정확도: 0.24025324891702765


* msbp_lgbm의 inference

In [50]:
inf_outputs = path2inf(outputs)
print(inf_outputs)

['baseline/output/densenet_e30_msbp.csv', 'baseline/output/resnet_e30_msbp.csv', 'baseline/output/resnet_e30_adam_msbp.csv']


In [52]:
msbp_inf_input = create_inf_data(inf_outputs)
msbp_inf_input.shape

(12600, 3)

In [53]:
msbp_inf_input[:10]

array([[13, 13,  0],
       [ 5,  1,  3],
       [13, 13,  1],
       [13, 13, 13],
       [12, 16,  0],
       [ 0,  1,  0],
       [ 2,  1,  7],
       [ 3,  4,  1],
       [ 4,  4,  4],
       [ 3,  3,  3]])

In [41]:
msbp_lgbm_inf = msbp_lgbm.predict(msbp_inf_input)

In [42]:
# basedataset으로만 구한 결과 csv
msbp_info = pd.read_csv('input/data/eval/info.csv')
msbp_info['ans'] = msbp_lgbm_inf
msbp_info.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,17


### 3) centerCrop - stacking

* 학습

In [84]:
outputs = glob.glob('baseline/train_output/*')
outputs = [file for file in outputs if file.endswith("centerCrop.csv")]
print(outputs)

['baseline/train_output/resnet_e30_msbp_centerCrop.csv', 'baseline/train_output/densenet_e30_msbp_centerCrop.csv']


In [85]:
cc_new_input = create_new_data(outputs)
cc_new_input.shape

(3001, 2)

In [86]:
cc_new_input[:10]

array([[ 1,  1],
       [ 0,  0],
       [12, 12],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 0,  0],
       [ 6,  6],
       [ 6,  0],
       [ 4,  0]])

In [87]:
df = pd.read_csv(outputs[0], names=['label','pred'])
labels = np.array(df['label'])
print(labels)

[ 0  0 12 ...  1  1  1]


In [88]:
# lgbm 학습
cc_lgbm = LGBMClassifier()

cc_lgbm.fit(cc_new_input, labels)
cc_lgbm_pred = cc_lgbm.predict(cc_new_input)

print(accuracy_score(labels, cc_lgbm_pred))

0.6321226257914029


In [89]:
print_acc(outputs, labels)

resnet_e30_msbp_centerCrop 정확도: 0.5454848383872043
densenet_e30_msbp_centerCrop 정확도: 0.5001666111296235


* centerCrop의 inference

In [90]:
inf_outputs = path2inf(outputs)
print(inf_outputs)

['baseline/output/resnet_e30_msbp_centerCrop.csv', 'baseline/output/densenet_e30_msbp_centerCrop.csv']


In [91]:
cc_inf_input = create_inf_data(inf_outputs)
cc_inf_input.shape

(12600, 2)

In [92]:
cc_inf_input[:10]

array([[3, 1],
       [3, 3],
       [0, 0],
       [0, 4],
       [3, 4],
       [3, 0],
       [3, 1],
       [4, 0],
       [3, 4],
       [3, 3]])

In [93]:
cc_lgbm_inf = cc_lgbm.predict(cc_inf_input)

In [94]:
# basedataset으로만 구한 결과 csv
cc_info = pd.read_csv('input/data/eval/info.csv')
cc_info['ans'] = cc_lgbm_inf
cc_info.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,3
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,3
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,4
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,3


#### 4) Hard Voting
* lgbm 모델이 두 개 뿐일 때 : 그냥 hard voting / stacking X

In [62]:
voting_lst = []
for i in range(len(base_inf_input)):
    voting_lst.append(list(base_inf_input[i]) + list(msbp_inf_input[i]))

In [64]:
voting_lst[:10]

[[13, 13, 13, 13, 0],
 [1, 5, 5, 1, 3],
 [6, 13, 13, 13, 1],
 [13, 13, 13, 13, 13],
 [12, 12, 12, 16, 0],
 [0, 0, 0, 1, 0],
 [1, 1, 2, 1, 7],
 [10, 4, 3, 4, 1],
 [4, 1, 4, 4, 4],
 [3, 3, 3, 3, 3]]

In [68]:
result = []
for data in voting_lst:
    result.append(max(data, key=data.count))

In [71]:
result[:10]

[13, 1, 13, 13, 12, 0, 1, 4, 4, 3]

* lgbm 모델이 두 개 이상일 때 : stacking 결과 -> hard voting

In [97]:
base_lgbm_inf

(12600,)

In [112]:
vote_lst = []
vote_lst.append(list(base_lgbm_inf))
vote_lst.append(list(msbp_lgbm_inf))
vote_lst.append(list(cc_lgbm_inf))
vote_lst = np.transpose(vote_lst)

In [113]:
vote_lst.shape

(12600, 3)

In [115]:
vote_lst[:10]

array([[13, 13,  3],
       [ 1,  2,  3],
       [13, 13,  0],
       [13, 13,  4],
       [12, 17,  3],
       [ 0,  0,  3],
       [ 1,  1,  3],
       [ 4,  3,  4],
       [ 1,  4,  3],
       [ 3,  3,  3]])

In [122]:
result = []
for data in list(vote_lst):
    result.append(max(data, key=list(data).count))

In [124]:
result[:10]

[13, 1, 13, 13, 12, 0, 1, 4, 1, 3]

## Soft Voting

In [69]:
outputs = glob.glob('baseline/output/*prob.csv')
print(outputs)
print(len(outputs))

['baseline/output/resnet_e30_adam_msbp_prob.csv', 'baseline/output/densenet_e30_msbp_centerCrop_prob.csv', 'baseline/output/densenet_e30_prob.csv', 'baseline/output/resnet18_e30_prob.csv', 'baseline/output/resnet_e30_msbp_prob.csv', 'baseline/output/densenet_e30_msbp_myAug_prob.csv']
6


In [9]:
for output in outputs:
    df = pd.read_csv(output)
    print(df.head())

In [91]:
df = pd.read_csv(output)
df.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,5
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12


In [40]:
def ans_list(model_idx, data_idx):
    df = pd.read_csv(outputs[model_idx])
    ans_list = list(np.float_(df['ans'][data_idx].split()[1:-1]))
    return ans_list

[3.948216, -1.5690122, -2.3456116, 1.143654, -2.9977727, -2.948003, -0.17715444, -3.5912628, -3.6973383, -2.3123062, -5.079523, -4.018175, 2.4623234, -1.2252645, -1.9448456, 0.42810258, -2.4179013, -2.478973]


## Hard Voting

In [2]:
outputs = glob.glob('baseline/output/*')
outputs = [file for file in outputs if not file.endswith("prob.csv")]
print(outputs)

['baseline/output/densenet_e30_msbp_myAug.csv', 'baseline/output/resnet18_e30.csv', 'baseline/output/resnet_e30_msbp.csv', 'baseline/output/resnet_e30_adam_msbp.csv', 'baseline/output/densenet_e30_msbp_centerCrop_0.05.csv', 'baseline/output/densenet_e30.csv']


In [3]:
# 각 모델의 decision 합치기 => new_input
new_input = []
for output in outputs:
    df = pd.read_csv(output)
    new_input.append(list(df['ans']))
new_input = np.transpose(new_input)
new_input.shape

(12600, 6)

In [4]:
new_input[:10]

array([[ 0, 13, 13,  0,  1, 13],
       [ 3,  1,  1,  3,  3,  5],
       [ 4,  6, 13,  1,  0, 13],
       [ 4, 13, 13, 13,  4, 13],
       [ 4, 12, 16,  0,  4, 12],
       [ 4,  0,  1,  0,  0,  0],
       [ 4,  1,  1,  7,  1,  1],
       [ 4, 10,  4,  1,  0,  4],
       [ 4,  4,  4,  4,  4,  1],
       [ 4,  3,  3,  3,  3,  3]])